<a href="https://colab.research.google.com/github/jjcheung/ctcuc22/blob/main/CheckDigitRevisionPractice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Submit this revision practice recording at https://forms.gle/SW3J4Y7LpfKWEbWm6 for 5% final assessment bonus credit.**

We have previously looked at check digit algorithm and validation for Singapore based NRIC (S & T) and FIN (F & G). (Assignment 3: https://colab.research.google.com/drive/13COimguGbZg-wF6fg4HslSRXXkIcTrz2?usp=sharing) 

"The Immigration & Checkpoints Authority (ICA) will introduce a new Foreign Identification Number (FIN) series with the prefix M from 1 January 2022. FINs are unique identification numbers issued by ICA and government agencies to foreigners who are working, studying, or residing in Singapore. Similar to the format of the F and G FIN series, the new M FIN series will start with the prefix M, followed by seven digits and a checksum letter (e.g. M1234567B)."

Source: https://www.ica.gov.sg/news-and-publications/newsroom/media-release/new-m-fin-series-to-be-introduced-from-1-january-2022

You also realised that https://samliew.com/nric-generator already caters for M-series FINs.


## Task 1
Extend your NRIC/FIN super validator so that it now caters to all NRICs and FINs starting with S, T, F, G and M.

In [ ]:
def generate_check_digit(nric):
    '''take in string and return check digit'''

    #definition of checksum and weights
    #checksum as dictionary of lists, with first letter of NRIC as keys and a list of check digits as values
    checksum = {                          
        'S': ['J', 'Z', 'I', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A'],
        'T': ['J', 'Z', 'I', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A'],
        'F': ['X', 'W', 'U', 'T', 'R', 'Q', 'P', 'N', 'M', 'L', 'K'],
        'G': ['X', 'W', 'U', 'T', 'R', 'Q', 'P', 'N', 'M', 'L', 'K'],
        'M': ['X', 'W', 'U', 'T', 'R', 'Q', 'P', 'N', 'M', 'L', 'K']
    }
    weights = [2, 7, 6, 5, 4, 3, 2]

    #algorithm to generate check digit
    total = 0
    for i in range (7):
        total += int(nric[i+1]) * weights[i]    
    if nric[0] in ['T', 'G']:             #additional step for T and G
        total += 4
    elif nric[0] == 'M':
        total += 3
    rmdr = total % 11                     #determining the index to the list in the dictionary

    return checksum[nric[0]][rmdr]        #returns checksum[key][value index], where key is first letter in NRIC, and rmdr is value index

def is_valid_check_digit(nric):
    '''to determine if check digit is valid'''

    #checking last digit in nric string with value returned from generate_check_digit()
    if nric[-1] == generate_check_digit(nric):
        print ('Valid NRIC/FIN check digit.')
    else:
        print ('Invalid NRIC/FIN check digit.')

def main():

    #checking valid input for NRIC
    valid_nric = False

    while not valid_nric:           #loops until input is valid
        snric = input('Enter NRIC/FIN number: ')
        if (len(snric) == 9) and (snric[0] in ['S', 'T', 'F', 'G', 'M']) and (snric[1:8].isnumeric()):
            #checks for length of input, valid first letter, and if 2nd to 8th character are all numerical values
            valid_nric = True       #ends loop
        else:                       #invalid entry
            print('Invalid input. Please key in again.')
    
    #calls check function
    is_valid_check_digit(snric)

#runs main()
main()

Enter NRIC/FIN number: M2068696T
Valid NRIC/FIN check digit.


## Task 2
You heard that [AI is the new electricity](https://www.gsb.stanford.edu/insights/andrew-ng-why-ai-new-electricity) and  everyone in town seems to be talking about machine learning. Assume you do not know the check digit algorithm for M-series FINs, but are able to generate enough valid M-series FINs using https://samliew.com/nric-generator. How would you design an algorithm and write program code to predict if a given M-series FIN is valid using a machine learning approach?


In [ ]:
#valid_mfins = ['M2068696T', 'M6081260P', 'M6973930R', 'M7667896T', 'M6611333N', 'M0964258N']

from scipy import stats
def getxy(string):
    checkdigit = ['X', 'W', 'U', 'T', 'R', 'Q', 'P', 'N', 'M', 'L', 'K']
    weights = [2, 7, 6, 5, 4, 3, 2]
    total = 0
    for i in range (7):
        total += int(string[i+1]) * weights[i]   
    rmdr = total % 11
    checki = checkdigit.index(string[-1])

    return [rmdr, checki]

def predicttrend(lst):
    x = []
    y = []

    for data in lst:
        result = getxy(data) 
        x.append(result[0]) #add rmdr to list of x
        y.append(result[1]) #add checki to list of y
    
    trend = stats.linregress(x,y) #determine slope and intercept fromn y = mx+c
    return trend

def generate_check_digit(nric, trend):
    '''take in string and return check digit'''

    #definition of checksum and weights
    #checksum as dictionary of lists, with first letter of NRIC as keys and a list of check digits as values
    checksum = ['X', 'W', 'U', 'T', 'R', 'Q', 'P', 'N', 'M', 'L', 'K']
    weights = [2, 7, 6, 5, 4, 3, 2]

    #algorithm to generate check digit
    total = 0
    for i in range (7):
        total += int(nric[i+1]) * weights[i]    
    rmdr = total % 11                     #determining the index to the list in the dictionary
    checki = int(trend.slope * rmdr + trend.intercept)
    return checksum[checki]       #returns checksum[key][value index], where key is first letter in NRIC, and rmdr is value index

def is_valid_check_digit(nric, trend):
    '''to determine if check digit is valid'''

    #checking last digit in nric string with value returned from generate_check_digit()
    if nric[-1] == generate_check_digit(nric, trend):
        print ('Valid FIN check digit.')
    else:
        print ('Invalid FIN check digit.')

def main():
    valid_mfins = ['M2068696T', 'M6081260P', 'M6973930R', 'M7667896T', 'M6611333N', 'M0964258N']

    trend = predicttrend(valid_mfins)
    
    #checking valid input for NRIC
    valid_nric = False

    while not valid_nric:           #loops until input is valid
        snric = input('Enter M FIN number: ')
        if (len(snric) == 9) and (snric[0] == 'M') and (snric[1:8].isnumeric()):
            #checks for length of input, valid first letter, and if 2nd to 8th character are all numerical values
            valid_nric = True       #ends loop
        else:                       #invalid entry
            print('Invalid input. Please key in again.')
    
    #calls check function
    is_valid_check_digit(snric, trend)
    
#runs main()
main()

Enter M FIN number: M2068696T
Valid FIN check digit.
